In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2022-02-07 16:42:13.953994: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-07 16:42:13.954031: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = pd.read_csv("Datasets/cardio_train1.csv")
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
data.isna().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [4]:
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]

In [5]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X = mms.fit_transform(X)
print(X[0])

[0.58807588 1.         0.57948718 0.27368421 0.01607916 0.01355014
 0.         0.         0.         0.         1.        ]


In [6]:
y = pd.get_dummies(y)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=321)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(11, activation='relu', input_shape=(11,)),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

2022-02-07 16:42:15.840747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-07 16:42:15.840782: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-07 16:42:15.840803: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (love-me): /proc/driver/nvidia/version does not exist
2022-02-07 16:42:15.841086: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                132       
                                                                 
 dense_1 (Dense)             (None, 30)                360       
                                                                 
 dense_2 (Dense)             (None, 2)                 62        
                                                                 
Total params: 554
Trainable params: 554
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

In [11]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=800)

Epoch 1/10
66/66 [==============================] - 1s 5ms/step - loss: 0.6791 - categorical_accuracy: 0.5728 - val_loss: 0.6623 - val_categorical_accuracy: 0.6177
Epoch 2/10
66/66 [==============================] - 0s 3ms/step - loss: 0.6540 - categorical_accuracy: 0.6264 - val_loss: 0.6436 - val_categorical_accuracy: 0.6342
Epoch 3/10
66/66 [==============================] - 0s 3ms/step - loss: 0.6421 - categorical_accuracy: 0.6362 - val_loss: 0.6371 - val_categorical_accuracy: 0.6377
Epoch 4/10
66/66 [==============================] - 0s 2ms/step - loss: 0.6373 - categorical_accuracy: 0.6392 - val_loss: 0.6345 - val_categorical_accuracy: 0.6387
Epoch 5/10
66/66 [==============================] - 0s 3ms/step - loss: 0.6346 - categorical_accuracy: 0.6418 - val_loss: 0.6324 - val_categorical_accuracy: 0.6414
Epoch 6/10
66/66 [==============================] - 0s 3ms/step - loss: 0.6332 - categorical_accuracy: 0.6410 - val_loss: 0.6311 - val_categorical_accuracy: 0.6432
Epoch 7/10
66/66

In [32]:
print(model.predict(X_test)[0])
y_test.head()

[0.31791398 0.68208605]


,0,1
59921,0,1
1843,1,0
25370,1,0
19006,1,0
41025,0,1


In [13]:
from sklearn.ensemble import RandomForestClassifier
Xr = data.iloc[:, 1:-1]
yr = data.iloc[:, -1]

In [14]:
rfc = RandomForestClassifier(n_estimators=200)
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, random_state=321)

In [15]:
rfc.fit(Xr_train, yr_train)

RandomForestClassifier(n_estimators=200)

In [16]:
from sklearn.metrics import classification_report

In [17]:
yr_pred = rfc.predict(Xr_test)
print(classification_report(yr_test, yr_pred))

              precision    recall  f1-score   support

           0       0.71      0.73      0.72      8782
           1       0.72      0.70      0.71      8718

    accuracy                           0.72     17500
   macro avg       0.72      0.72      0.72     17500
weighted avg       0.72      0.72      0.72     17500

